In [6]:
import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn

In [7]:
class StandardScaler:

    def __init__(self, mean=None, std=None, epsilon=1e-7):
     
        self.mean = mean
        self.std = std
        self.epsilon = epsilon

    def fit(self, values):
        dims = list(range(values.dim() - 1))
        self.mean = torch.mean(values, dim=dims)
        self.std = torch.std(values, dim=dims)

    def transform(self, values):
        return (values - self.mean) / (self.std + self.epsilon)

    def fit_transform(self, values):
        self.fit(values)
        return self.transform(values)

    def __repr__(self):
        return f"mean: {self.mean}, std:{self.std}, epsilon:{self.epsilon}"

In [8]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(in_features=50, out_features=3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

In [9]:
irisDatasetCinco = pd.read_csv("iris_cinco.data", 
                          names=["sepallength","sepalwidth",
                                 "petallength","petalwidth"])

s1=irisDatasetCinco.iloc[:,0:4].values
x_tensor = torch.tensor(s1)
scaler = StandardScaler(mean = torch.tensor([5.84, 3.05, 3.76, 1.20]), std = torch.tensor([0.83, 0.43, 1.76, 0.76]))
XScalada = scaler.fit_transform(x_tensor).type(torch.float32)

model = Model(4)

model.load_state_dict(torch.load("pesos.pt"))
model.eval()
torch.set_printoptions(precision=6, sci_mode=False)
saida = model(XScalada)
print(saida)

tensor([[    0.999881,     0.000119,     0.000000],
        [    0.998057,     0.001943,     0.000000],
        [    0.000088,     0.503979,     0.495934],
        [    0.000000,     0.000002,     0.999998],
        [    0.000067,     0.999548,     0.000385]],
       grad_fn=<SoftmaxBackward0>)
